In [16]:
import os
import sys
import json
import itertools
import PIL.Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from itertools import combinations
from scipy.stats import shapiro, f_oneway, kruskal
from sklearn.metrics import classification_report, confusion_matrix
from statsmodels.stats.multicomp import MultiComparison

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(os.path.join(module_path))

from functions.ciou import ciou_loss, ciou_metric
from functions.loading_data import SMALLER_HEIGHT, SMALLER_WIDTH
from functions.plotting import plot_bars, plot_bar_from_dict

WIDTH = 200
HEIGHT = 150

In [17]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', '..', '..', 'data1', 'images_original_inception_resnet_v2_200x150_splitted')
test_dir = os.path.join(base_dir, 'validation')

In [18]:
ROOT_NAME = 'custom_models'
models_dir = os.path.join(
    '..',
    '..',
    'models',
    'data1_roi')
model_names = list(filter(lambda model_file_name: ROOT_NAME in model_file_name, os.listdir(models_dir)))

In [19]:
accuracies_data_path = os.path.join('..', '..', 'histories', f'{ROOT_NAME}_accuracies.json')
losses_data_path = os.path.join('..', '..', 'histories', f'{ROOT_NAME}_losses.json')

In [20]:
def get_images_array(paths: list[str]) -> np.ndarray:
    rows = []
    rescale = keras.layers.Rescaling(1./255)

    for path in paths:
        with PIL.Image.open(path) as image:
            image_array = np.asarray(image)
            rescaled_image = rescale(image_array)
            rows.append(rescaled_image)

    return np.array(rows)


test_paths = get_paths(test_dir)
X_test = get_images_array(test_paths)

In [21]:
test_names = set(get_names(test_dir))
metadata_path = os.path.join('..', '..', '..', 'data1', 'HAM10000_metadata_ext.csv')
data = pd.read_csv(metadata_path).sort_values(by='image_id')
relevant_cols = ['top', 'left', 'bottom', 'right']
test_df = data[data['image_id'].isin(test_names)][relevant_cols]
ys_test = test_df.to_numpy().astype(float)
ys_test[:, [0, 2]] /= SMALLER_HEIGHT
ys_test[:, [1, 3]] /= SMALLER_WIDTH

In [22]:
def get_accuracies_and_losses(models: list[str]):
    results = []

    for model_name in models:
        model_path = os.path.join(models_dir, model_name)
        model = keras.models.load_model(model_path, compile=False, custom_objects={'loss': ciou_loss, 'ciou_metric': ciou_metric})

        if 'multiattentive_model' not in model_name:
            model.compile(optimizer='adam', loss=ciou_loss, metrics=[ciou_metric])
        else:
            model.compile(
                optimizer='adam',
                loss={
                    'root': ciou_loss,
                    'dot': keras.losses.mean_squared_error,
                },
                loss_weights={'root': 1.0, 'dot': 0.1},
                metrics={
                    'root': ciou_metric,
                    'dot': keras.metrics.mean_squared_error
                })

        eval_result = model.evaluate(X_test, ys_test)

        results.append(eval_result)

    return results


# if not os.path.exists(accuracies_data_path):
resx = get_accuracies_and_losses(model_names)

print(resx)

    # with open(accuracies_data_path, 'w') as json_file:
    #     json.dump(accuracies, json_file, indent=4)

63/63 [==============================] - 2s 21ms/step - loss: 0.6110 - ciou_metric: 0.3890
[[0.6081937551498413, 0.3918073773384094], [0.606543779373169, 0.39351266622543335], [0.6083054542541504, 0.39170747995376587], [0.6066533327102661, 0.39334893226623535], [0.6110488176345825, 0.3890065550804138]]
